In [10]:
password = 'js0815' # 본인의 DB password

In [11]:
import pandas as pd
import numpy as np
import time, json, requests 
from datetime import datetime, timedelta, date

import pymysql # python에서 mysql을 사용하는 패키지

import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine

In [13]:
server = '127.0.0.1:3306' # local server
user = 'root' # user name
password = password # 개인 password
db = '16010980_my1st_db' # DB 이름

# sqlalchemy의 create_engine을 이용해 DB 연결
engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))

In [14]:
def getDateRange(st_date, end_date):
    print(st_date)
    print(int((end_date - st_date).days)+1)
    for n in range(int((end_date - st_date).days)+1):
        #print(st_date + timedelta(days=n))
        yield st_date + timedelta(days=n)

In [15]:
def getKRXPrice(mktId, st_dt, end_dt):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020103"
    }

    # 수집 기간 설정
    sdate = datetime.strptime(st_dt,'%Y%m%d').date()
    edate = datetime.strptime(end_dt,'%Y%m%d').date()
    dt_idx = []
    for dt in getDateRange(sdate, edate):
        if dt.isoweekday() < 6:
            dt_idx.append(dt.strftime("%Y%m%d"))

    daily = []
    for dt in dt_idx:
        # 날짜별 전종목 일봉 불러오기
        p_data = {
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',
            'mktId': mktId,
            'trdDd': dt,
            'share': '1',
            'money': '1',
            'csvxls_isNo': 'false'
        }

        url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
        res = requests.post(url, headers=headers, data=p_data)
        html_text = res.content
        html_json = json.loads(html_text)
        html_jsons = html_json['OutBlock_1']

        if len(html_jsons) > 0:
            for html_json in html_jsons:
                if html_json['TDD_OPNPRC'] == '-': # 시장이 열리지 않아 값이 없는 경우
                    continue

                ISU_SRT_CD = html_json['ISU_SRT_CD']
                ISU_ABBRV = html_json['ISU_ABBRV']
                TRD_DD = datetime.strptime(dt,'%Y%m%d').strftime('%Y-%m-%d')

                FLUC_RT = float(html_json['FLUC_RT'].replace(',',''))/100
                TDD_CLSPRC = int(html_json['TDD_CLSPRC'].replace(',',''))
                TDD_OPNPRC = int(html_json['TDD_OPNPRC'].replace(',',''))
                TDD_HGPRC = int(html_json['TDD_HGPRC'].replace(',',''))
                TDD_LWPRC = int(html_json['TDD_LWPRC'].replace(',',''))

                ACC_TRDVOL = int(html_json['ACC_TRDVOL'].replace(',',''))
                ACC_TRDVAL = int(html_json['ACC_TRDVAL'].replace(',',''))
                MKTCAP = int(html_json['MKTCAP'].replace(',',''))
                LIST_SHRS = int(html_json['LIST_SHRS'].replace(',',''))

                daily.append((ISU_SRT_CD,ISU_ABBRV,TRD_DD,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,TDD_CLSPRC,ACC_TRDVOL,FLUC_RT,ACC_TRDVAL,MKTCAP,LIST_SHRS))
        else:
            pass

    if len(daily) > 0:
        daily = pd.DataFrame(daily)
        daily.columns = ['stock_code','stock_name','date','open','high','low','close','volume','change','ACC_TRDVAL','MKTCAP','LIST_SHRS']
        daily = daily.sort_values(by='date').reset_index(drop=True)
        return daily

    else:
        return pd.DataFrame()

In [16]:
pr_df = getKRXPrice('STK', '20220320', '20220401')

2022-03-20
13


In [17]:
pr_df

,stock_code,stock_name,date,open,high,low,close,volume,change,ACC_TRDVAL,MKTCAP,LIST_SHRS
0,095570,AJ네트웍스,2022-03-21,5590,5930,5580,5800,195418,0.0526,1126976550,271569311000,46822295
1,000100,유한양행,2022-03-21,59600,59600,57500,58600,169153,-0.0201,9921240800,4291364493600,73231476
2,000105,유한양행우,2022-03-21,57200,57300,56400,56800,1298,-0.0070,74000600,67077392000,1180940
3,003460,유화증권,2022-03-21,2545,2545,2515,2545,59364,0.0000,150720235,144307646175,56702415
4,003465,유화증권우,2022-03-21,2615,2620,2615,2620,1550,0.0019,4055870,45788233500,17476425
...,...,...,...,...,...,...,...,...,...,...,...,...
9391,014820,동원시스템즈,2022-04-01,58400,58500,57200,57300,24448,-0.0272,1408944300,1672495950300,29188411
9392,014825,동원시스템즈우,2022-04-01,35350,35550,35000,35400,634,0.0028,22493350,9393673200,265358
9393,163560,동일고무벨트,2022-04-01,7200,7380,7150,7340,20198,0.0166,146833710,102026000000,13900000
9394,001529,동양3우B,2022-04-01,36300,37500,36300,37400,13219,-0.0273,487597750,3355602800,89722


In [18]:
pr_df.to_sql(name='kospi_unadj',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                 'stock_code' : sqlalchemy.types.VARCHAR(10),
                 'stock_name' : sqlalchemy.types.TEXT(),
                 'date' : sqlalchemy.types.DATE(),
                 'open' : sqlalchemy.types.BIGINT(),
                 'high' : sqlalchemy.types.BIGINT(),
                 'low' : sqlalchemy.types.BIGINT(),
                 'close' : sqlalchemy.types.BIGINT(),
                 'volume' : sqlalchemy.types.BIGINT(),
                 'change' : sqlalchemy.types.FLOAT(),
                 'ACC_TRDVAL' : sqlalchemy.types.BIGINT(),
                 'MKTCAP' : sqlalchemy.types.BIGINT(),
                 'LIST_SHRS' : sqlalchemy.types.BIGINT()                 
             }
            )

In [19]:
# DB 접속 해제
engine.dispose()
#cur.close()
#conn.close()